In [1]:
import requests
from bs4 import BeautifulSoup

r = requests.get("http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c = r.content

soup = BeautifulSoup(c, "html.parser")

all = soup.find_all("div", {"class":"propertyRow"}) # returns all the elements with div class, you may check each element

In [8]:
# getting the price of property in the first row 
all[0].find("h4", {"class":"propPrice"}).text.replace("\n","").replace(" ", "") # replaces all \n and whitespace with ""

'$725,000'

In [11]:
# getting the address of property in the first row 
all[0].find("span", {"class":"propAddressCollapse"}).text

'0 Gateway'

In [12]:
# getting the address of property in the first row 
all[1].find("span", {"class":"propAddressCollapse"}).text

'1003 Winchester Blvd.'

In [13]:
# getting the number of beds of property in the 2nd row since 1st row is None 
all[1].find("span", {"class":"infoBed"}).find("b").text

'4'

In [14]:
# getting the number of baths of property in the 2nd row since 1st row is None 
all[1].find("span", {"class":"infoValueFullBath"}).find("b").text

'4'

In [15]:
# getting the area of property in the 3rd row since 1st & 2nd row is None 
all[2].find("span", {"class":"infoSqFt"}).find("b").text

'3,154'

In [2]:
# iterate in  for loop all elements you want to get and assign it to a dictionary
# note: for the ff elements: Beds, Area, Full Baths and Half Baths, I converted the text to int 
# so that whenever it is saved into the excel file, those will be saved as numbers

list_features = []


for i in all:
    d = {}
    d["Address"] = i.find_all("span", {"class":"propAddressCollapse"})[0].text
    d["Locality"] = i.find_all("span", {"class":"propAddressCollapse"})[1].text
    d["Price"] = i.find("h4", {"class":"propPrice"}).text.replace("\n", "").replace(" ", "")
    
    try:
        d["Beds"] = int(i.find("span", {"class":"infoBed"}).find("b").text)
    except:
        d["Beds"] = None
    
    try:
        d["Area"] = int(i.find("span", {"class":"infoSqFt"}).find("b").text)
    except:
        d["Area"] = None
    
    try:
        d["Full Baths"] = int(i.find("span", {"class":"infoValueFullBath"}).find("b").text)
    except:
        d["Full Baths"] = None
    
    try:
        d["Half Baths"] = int(i.find("span", {"class":"infoValueHalfBath"}).find("b").text)
    except:
        d["Half Baths"] = None
       
    try:
        for column_group in i.find_all("div", {"class":"columnGroup"}):
            for feature_group, feature_name in zip(column_group.find_all("span", {"class":"featureGroup"}),
                                               column_group.find_all("span", {"class": "featureName"})):
                if "Lot Size" in feature_group.text:
                    d["Lot Size"] = feature_name.text
    except:
        print(None)
    
    list_features.append(d)

In [58]:
list_features

[{'Address': '0 Gateway',
  'Locality': 'Rock Springs, WY 82901',
  'Price': '$725,000',
  'Beds': None,
  'Area': None,
  'Full Baths': None,
  'Half Baths': None},
 {'Address': '1003 Winchester Blvd.',
  'Locality': 'Rock Springs, WY 82901',
  'Price': '$452,900',
  'Beds': 4,
  'Area': None,
  'Full Baths': 4,
  'Half Baths': None,
  'Lot Size': '0.21 Acres'},
 {'Address': '600 Talladega',
  'Locality': 'Rock Springs, WY 82901',
  'Price': '$396,900',
  'Beds': 5,
  'Area': None,
  'Full Baths': 3,
  'Half Baths': None},
 {'Address': '3239 Spearhead Way',
  'Locality': 'Rock Springs, WY 82901',
  'Price': '$389,900',
  'Beds': 4,
  'Area': None,
  'Full Baths': 3,
  'Half Baths': 1,
  'Lot Size': 'Under 1/2 Acre, '},
 {'Address': '522 Emerald Street',
  'Locality': 'Rock Springs, WY 82901',
  'Price': '$254,000',
  'Beds': 3,
  'Area': None,
  'Full Baths': 3,
  'Half Baths': None,
  'Lot Size': 'Under 1/2 Acre, '},
 {'Address': "1302 Veteran's Drive",
  'Locality': 'Rock Springs, W

In [3]:
# store the above list in a dataframe

import pandas as pd

df = pd.DataFrame(list_features)
df.to_excel("webscraping_output.xlsx")